## Identifying the Type of a Lambda Expression

* type is known at compile time
* the type of a lambda expression could be:
    - the type of a variable
    - field
    - method parameter
    - or returned type of a method
* there is a restriction on the type of a lambda expression
    - it has to be a functional interface
    - an anonymous class that does not implement a functional interface cannot be written as a lambda expression
    - __functional interface: the simplest definition is an interface that has one and only one _abstract_ method regardless of the number of default or static concrete methods inside it__
        * so an inteface with only a default method or static method cannot be a functional interface
* note: a functional interface does not require the annotation @FunctionalInterface
    - it's just there to make sure that your interface is indeed functional
    - if you put it on a type that isn't, then the compiler will raise an error

### Examples of Functional Interfaces

In [ ]:
// Runnable is functional
// only has 1 abstract method
// @FunctionalInterface annotation is optional, it's only there as a helper to check

@FunctionalInterface
public interface Runnable {
    public abstract void run();
}

In [ ]:
// Consumer<T> is functional
// has exactly 1 abstract method
// and 1 default concrete method that doesn't count

@FunctionalInterface
public interface Consumer<T> {

    void accept(T t);

    default Consumer<T> andThen(Consumer<? super T> after) {
        // the body of this method has been removed
    }
}


In [ ]:
// Predicate<T> is functional
// it has exactly 1 abstract method
// 3 default methods that don't count
// and 2 static methods that don't count

@FunctionalInterface
public interface Predicate<T> {

    boolean test(T t);

    default Predicate<T> and(Predicate<? super T> other) {
        // the body of this method has been removed
    }

    default Predicate<T> negate() {
        // the body of this method has been removed
    }

    default Predicate<T> or(Predicate<? super T> other) {
        // the body of this method has been removed
    }

    static <T> Predicate<T> isEqual(Object targetRef) {
        // the body of this method has been removed
    }

    static <T> Predicate<T> not(Predicate<? super T> target) {
        // the body of this method has been removed
    }
}


## Finding the Right Method to Implement

* a lambda expression is an implementation of the only abstract method in the functional interface
    - so just look for them in the interface

In [ ]:
// Runnable interface
public abstract void run();

// Predicate interface
boolean test(T t);

// Consumer interface
void accept(T t);

## Implementing the Right Method with a Lambda Expression

### Writing a First Lambda Expression that Implements _Predicate\<String\>_

* syntax is made of 3 elements:
    1. block of parameters
    2. little piece of ASCII art depicting an arrow: ->
        - Java uses meager arrows (->) and not fat arrows (=>)
    3. block of code which is the body of the method
* suppose you need an instance of a Predicate that returns true for strings of characters that have exactly 3 characters:
    1. the type of your lambda expression is Predicate
    2. the method you need to implement is boolean test(String s)

In [ ]:
Predicate<String> predicate =
    // same method signature as test() in Predicate<T>
    (String s) -> {
        return s.length() == 3;
    };

### Simplifying the Syntax

* (String s) can be simplified to s
    - the compiler knows that this abstract method takes String as an argument so you can skip declaring the type
    - and since there's only 1 parameter, you can remove the parentheses
* there's only 1 line of code in the body of the method
    - thus, you can ignore the curly braces and remove the return keyword
* good practice: keep your lambdas short so that they are just one line of simple, readable code

In [ ]:
Predicate<String> predicate = s -> s.length() == 3;

### Implementing a Consumer\<String\>

* lambda that consumes a string and prints on System.out

In [ ]:
Consumer<String> print = s -> System.out.println(s);

### Implementing a Runnable

* writes out an implementation of void run()
* block of arguments is empty so should be written with empty parentheses

In [ ]:
Runnable runnable = () -> System.out.println("I am running");

## Calling a Lambda Expression

* use the variable that the lambda expression was assigned to
* remember that a lambda expression is an implementation of an abstract method from a functional interface
    - therefore, you can use the variable, call the abstract method directly, and pass in any arguments to it which would call the lambda expression
    - thus, calling the abstract method will call the code of your lambda itself

In [ ]:
List<String> retainStringsOfLength3(List<String> strings) {

    Predicate<String> predicate = s -> s.length() == 3;
    List<String> stringsOfLength3 = new ArrayList<>();
    for (String s: strings) {
    
        // calling the lambda expression
        // keep in mind, a lambda expression is an implementation of an abstract method from a functional interface
        if (predicate.test(s)) {
            stringsOfLength3.add(s);
        }
    }
    return stringsOfLength3;
}


## Capturing Local Values

* in the example below:
    - compiling it will give you an error on the use of totalPrice in the Consumer implementation
        * error: Variable used in lambda expression should be final or effectively final
    - reason being, lambdas cannot modify variables defined outside their body
        * they can read them, as long as they are final, that is, immutable
* the process of accessing variable is called _capturing_
    - lambdas cannot capture variables, they can only capture values
    - and that's why they can read variables that are final
        * _a final variable is a value_
* note that the error message tells us that the variable can be final or effectively final
    - even if you do not explicitly declare a variable final, the compiler may do it for you, making it effectively final
    - if the compiler sees that this variable is read from a lambda and you do not modify it, it will nicely add the final declaration for you
    - this is done in the compiled code and does not modify the source code

In [ ]:
int calculateTotalPrice(List<Product> products) {

    int totalPrice = 0;
    Consumer<Product> consumer =
        product -> totalPrice += product.getPrice();
    for (Product product: products) {
        consumer.accept(product);
    }
}

## Serializing Lambdas

* why serialize lambda expressions?
    - lambdas can be stored in fields and this field may be accessed through a constructor or a setter method
    - then you may have a lambda in the state of your object at runtime, without being aware of it
    - thus, to preserve backward compatibility with existing serializable classes, serializing a lambda expression is possible